In [ ]:
#TQC:https://github.com/brickerino/ada-tqc/blob/master/tqc/structures.py
#ADA:https://github.com/SamsungLabs/tqc_pytorch/blob/master/main.py
#ACC:https://github.com/Nicolinho/ACC

In [1]:
!sudo apt update
!sudo apt install xvfb -y
!apt-get install python-opengl -y
!pip install 'swig'
!pip install 'pyglet==1.5.11'
!pip install 'gym[box2d]==0.20.0'
!pip install 'pyvirtualdisplay==3.0'
!pip install matplotlib
import gym
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import sys
from pyvirtualdisplay import Display
from IPython import display as disp
%matplotlib inline
#from xvfbwrapper import Xvfb
#xvfb-run -s
display = Display(visible=0,size=(600,600))
display.start()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

plot_interval = 10 # update the plot every N episodes
video_every = 100 # videos can take a very long time to render so only do it every N episode

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1581 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]    
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done0m0m
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't 

In [2]:


import gym
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import sys
from pyvirtualdisplay import Display
from IPython import display as disp
%matplotlib inline

display = Display(visible=0,size=(600,600))
display.start()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

plot_interval = 10 # update the plot every N episodes
video_every = 100 # videos can take a very long time to render so only do it every N episodes

In [3]:
    def states_by_ptr(self, ptr_list, cpu=False):
        ind = np.array([], dtype='int64')
        for interval in ptr_list:
            if interval[0] < interval[1]:
                ind = np.concatenate((ind, np.arange(interval[0], interval[1])))
            elif interval[0] > interval[1]:
                ind = np.concatenate((ind, np.arange(interval[0], self.max_size)))
                ind = np.concatenate((ind, np.arange(0, interval[1])))

        names = ('state', 'action')
        if cpu:
            return (torch.FloatTensor(getattr(self, name)[ind]) for name in names)
        else:
            return (torch.FloatTensor(getattr(self, name)[ind]).to(DEVICE) for name in names)

In [4]:
import numpy as np

import torch

DEVICE = device


def eval_policy(policy, eval_env, max_episode_steps, eval_episodes=10):
    policy.eval()
    avg_reward = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        t = 0
        while not done and t < max_episode_steps:
            action = policy.select_action(state)
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward
            t += 1
    avg_reward /= eval_episodes
    policy.train()
    return avg_reward

def eval_bias(policy, critic, eval_env, gamma, max_episode_steps, extra_steps, eval_episodes, target_entropy, alpha):
    policy.train()
    all_values = []
    all_returns = []
    for _ in range(eval_episodes):
        ep_rewards = []
        ep_states = []
        ep_actions = []
        state, done = eval_env.reset(), False
        t = 0
        while not done and t < max_episode_steps + extra_steps:
            action = policy.select_action(state)
            ep_states.append(state)
            ep_actions.append(action)
            state, reward, done, _ = eval_env.step(action)
            ep_rewards.append(reward)
            t += 1

        cur_return = 0
        returns = []
        for r in ep_rewards[::-1]:
            cur_return = r + alpha * target_entropy + cur_return * gamma
            returns.append(cur_return)
        returns = returns[::-1]

        n_to_use = min(max_episode_steps, t)
        returns_to_use = np.array(returns[:n_to_use])
        all_returns.append(returns_to_use)

        states_to_use = np.stack(ep_states[:n_to_use])
        states_to_use_torch = torch.FloatTensor(states_to_use).to(DEVICE)
        actions_to_use = np.stack(ep_actions[:n_to_use])
        actions_to_use_torch = torch.FloatTensor(actions_to_use).to(DEVICE)
        with torch.no_grad():
            values = critic(states_to_use_torch, actions_to_use_torch).mean(axis=2).mean(axis=1)
            values = values.cpu().numpy()
        all_values.append(values)
    return all_values, all_returns


def quantile_huber_loss_f(quantiles, samples, samples_mask=None):
    pairwise_delta = samples[:, None, None, :] - quantiles[:, :, :, None]  # batch x nets x quantiles x samples
    abs_pairwise_delta = torch.abs(pairwise_delta)
    huber_loss = torch.where(abs_pairwise_delta > 1,
                             abs_pairwise_delta - 0.5,
                             pairwise_delta ** 2 * 0.5)

    n_quantiles = quantiles.shape[2]
    tau = torch.arange(n_quantiles, device=DEVICE).float() / n_quantiles + 1 / 2 / n_quantiles
    loss = (torch.abs(tau[None, None, :, None] - (pairwise_delta < 0).float()) * huber_loss)
    if samples_mask is not None:
        samples_mask = samples_mask.unsqueeze(0).unsqueeze(0)
        loss = (loss * samples_mask).sum(-1) / samples_mask.sum()
    loss = loss.mean()
    return loss

In [5]:
import numpy as np
import torch
from torch.nn import Module, Linear
from torch.distributions import Distribution, Normal
from torch.nn.functional import relu, logsigmoid
from gym import spaces
import gym
from collections import deque
from math import pow



LOG_STD_MIN_MAX = (-20, 2)


class RescaleAction(gym.ActionWrapper):
    def __init__(self, env, a, b):
        assert isinstance(env.action_space, spaces.Box), (
            "expected Box action space, got {}".format(type(env.action_space)))
        assert np.less_equal(a, b).all(), (a, b)
        super(RescaleAction, self).__init__(env)
        self.a = np.zeros(env.action_space.shape, dtype=env.action_space.dtype) + a
        self.b = np.zeros(env.action_space.shape, dtype=env.action_space.dtype) + b
        self.action_space = spaces.Box(low=a, high=b, shape=env.action_space.shape, dtype=env.action_space.dtype)

    def action(self, action):
        assert np.all(np.greater_equal(action, self.a)), (action, self.a)
        assert np.all(np.less_equal(action, self.b)), (action, self.b)
        low = self.env.action_space.low
        high = self.env.action_space.high
        action = low + (high - low)*((action - self.a)/(self.b - self.a))
        action = np.clip(action, low, high)
        return action


class Mlp(Module):
    def __init__(
            self,
            input_size,
            hidden_sizes,
            output_size
    ):
        super().__init__()
        # TODO: initialization
        self.fcs = []
        in_size = input_size
        for i, next_size in enumerate(hidden_sizes):
            fc = Linear(in_size, next_size)
            self.add_module(f'fc{i}', fc)
            self.fcs.append(fc)
            in_size = next_size
        self.last_fc = Linear(in_size, output_size)

    def forward(self, input):
        h = input
        for fc in self.fcs:
            h = relu(fc(h))
        output = self.last_fc(h)
        return output


class ReplayBuffer(object):
    def __init__(self, state_dim, action_dim, max_size=int(1e6), gamma=0.99, n_episodes_to_store=50, q_g_rollout_length=None):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.gamma = gamma
        self.n_episodes_to_store = n_episodes_to_store
        self.q_g_rollout_length = q_g_rollout_length

        self.transition_names = ('state', 'action', 'next_state', 'reward', 'not_done', 'ep_end',
                                 'returns', 'ep_length', 'bs_multiplier')
        sizes = (state_dim, action_dim, state_dim, 1, 1, 1, 1, 1, 1)
        for name, size in zip(self.transition_names, sizes):
            setattr(self, name, np.empty((max_size, size)))

        self.last_episodes = deque()

    def add(self, state, action, next_state, reward, done, ep_end):
        values = (state, action, next_state, reward, 1. - done, ep_end)
        for name, value in zip(self.transition_names, values):
            getattr(self, name)[self.ptr] = value

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

        if ep_end:
            res_idx = []    # indices to put into fresh replay buffer
            running_return = 0
            running_tail_return = 0
            was_timelimit = self.not_done[(self.ptr - 1) % self.max_size, 0] > 0.5
            for t in range(self.size):
                idx = (self.ptr - 1 - t) % self.max_size   # index of tuple in replay
                if t > 0 and (self.ep_end[idx, 0] > 0.5):
                    break                                  # got to the previous episode end
                running_return = self.reward[idx] + self.gamma * running_return       # running return at current index
                if t >= self.q_g_rollout_length:
                    running_tail_return = self.reward[(idx + self.q_g_rollout_length) % self.max_size] + self.gamma * running_tail_return
                self.returns[idx] = running_return - np.power(self.gamma, self.q_g_rollout_length) * running_tail_return       # only indices in between
                self.ep_length[idx] = t + 1

                is_enough_rollout = t + 1 > self.q_g_rollout_length
                if (was_timelimit and is_enough_rollout) or not was_timelimit:
                    res_idx.append(idx)
                    if is_enough_rollout:
                        self.bs_multiplier[idx] = 1.0
                    else:
                        self.bs_multiplier[idx] = 0.0
            if len(res_idx) > 5:
                self.last_episodes.append(np.array(res_idx, dtype='int32'))
                if len(self.last_episodes) > self.n_episodes_to_store:
                    self.last_episodes.popleft()

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        names = self.transition_names[:-2]
        return (torch.FloatTensor(getattr(self, name)[ind]).to(DEVICE) for name in names)

    def gather_returns(self, gamma, alpha, n_per_episode):
        selected_idx = []
        for ep_lst in self.last_episodes:
            selected_idx.append(np.random.choice(ep_lst, replace=True, size=n_per_episode))
        selected_idx = np.concatenate(selected_idx)
        return self.get_returns_by_idx(selected_idx, gamma, alpha)

    def gather_returns_uniform(self, gamma, alpha, n_per_episode):
        all_idx = np.concatenate(self.last_episodes)
        selected_idx = np.random.choice(all_idx, replace=True, size=n_per_episode * len(self.last_episodes))
        return self.get_returns_by_idx(selected_idx, gamma, alpha)

    def get_returns_by_idx(self, selected_idx, gamma, alpha):
        h_target = - self.action.shape[1]
        result_states = self.state[selected_idx]
        result_actions = self.action[selected_idx]
        result_bs_multiplier = self.bs_multiplier[selected_idx]
        result_bs_states = self.state[(selected_idx + self.q_g_rollout_length) % self.max_size]         # if bs_multiplier is 0 non-existing states should not matter
        ep_length = self.ep_length[selected_idx]
        ns = (ep_length - 1) * (1 - result_bs_multiplier) + self.q_g_rollout_length * result_bs_multiplier
        entropy_сorrection = alpha * h_target * gamma * (1 - np.power(gamma, ns)) / (1 - gamma)
        result_returns = self.returns[selected_idx] + entropy_сorrection
        return (torch.tensor(arr, dtype=torch.float32, device=DEVICE) for arr in
                [result_states, result_actions, result_returns, result_bs_states, result_bs_multiplier])

    def states_by_ptr(self, ptr_list, cpu=False):
        ind = np.array([], dtype='int64')
        for interval in ptr_list:
            if interval[0] < interval[1]:
                ind = np.concatenate((ind, np.arange(interval[0], interval[1])))
            elif interval[0] > interval[1]:
                ind = np.concatenate((ind, np.arange(interval[0], self.max_size)))
                ind = np.concatenate((ind, np.arange(0, interval[1])))

        names = ('state', 'action')
        return (torch.FloatTensor(getattr(self, name)[ind]).to(DEVICE) for name in names)

class Critic(Module):
    def __init__(self, state_dim, action_dim, n_quantiles, n_nets):
        super().__init__()
        self.nets = []
        self.n_quantiles = n_quantiles
        self.n_nets = n_nets
        for i in range(n_nets):
            net = Mlp(state_dim + action_dim, [512, 512, 512], n_quantiles)
            self.add_module(f'qf{i}', net)
            self.nets.append(net)

    def forward(self, state, action):
        sa = torch.cat((state, action), dim=1)
        quantiles = torch.stack(tuple(net(sa) for net in self.nets), dim=1)
        return quantiles


class Agent(Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.action_dim = action_dim
        self.net = Mlp(state_dim, [256, 256], 2 * action_dim)

    def forward(self, obs):
        mean, log_std = self.net(obs).split([self.action_dim, self.action_dim], dim=1)
        log_std = log_std.clamp(*LOG_STD_MIN_MAX)

        if self.training:
            std = torch.exp(log_std)
            tanh_normal = TanhNormal(mean, std)
            action, pre_tanh = tanh_normal.rsample()
            log_prob = tanh_normal.log_prob(pre_tanh)
            log_prob = log_prob.sum(dim=1, keepdim=True)
        else:  # deterministic eval without log_prob computation
            action = torch.tanh(mean)
            log_prob = None
        return action, log_prob

    def sample_action(self, obs):
        obs = torch.FloatTensor(obs).to(DEVICE)[None, :]
        action, _ = self.forward(obs)
        action = action[0].cpu().detach().numpy()
        return action


class TanhNormal(Distribution):
    def __init__(self, normal_mean, normal_std):
        super().__init__()
        self.normal_mean = normal_mean
        self.normal_std = normal_std
        self.standard_normal = Normal(torch.zeros_like(self.normal_mean, device=DEVICE),
                                      torch.ones_like(self.normal_std, device=DEVICE))
        self.normal = Normal(normal_mean, normal_std)

    def log_prob(self, pre_tanh):
        log_det = 2 * np.log(2) + logsigmoid(2 * pre_tanh) + logsigmoid(-2 * pre_tanh)
        result = self.normal.log_prob(pre_tanh) - log_det
        return result

    def rsample(self):
        pretanh = self.normal_mean + self.normal_std * self.standard_normal.sample()
        return torch.tanh(pretanh), pretanh

In [6]:
import torch


import numpy as np
import torch
from math import ceil, floor



class Trainer(object):
    def __init__(
        self,
        *,
        actor,
        critic,
        critic_target,
        discount,
        tau,
        top_quantiles_to_drop,
        target_entropy,
        sampling_scheme,
        Q_G_eval_interval,
        Q_G_n_per_episode,
        delta_gamma,
        d_update_interval,use_acc,
                      lr_dropped_quantiles,
                      adjusted_dropped_quantiles_init,
                      adjusted_dropped_quantiles_max,
                      diff_ma_coef,
                      num_critic_updates
    ):
        self.actor = actor
        self.critic = critic
        self.critic_target = critic_target
        self.log_alpha = torch.zeros((1,), requires_grad=True, device=DEVICE)

        # TODO: check hyperparams
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)
        self.alpha_optimizer = torch.optim.Adam([self.log_alpha], lr=3e-4)

        self.discount = discount
        self.tau = tau
        self.top_quantiles_to_drop = top_quantiles_to_drop
        self.target_entropy = target_entropy

        self.quantiles_total = critic.n_quantiles * critic.n_nets
        self.samples_mask = torch.zeros(1, self.quantiles_total, dtype=torch.float32, device=DEVICE)
        self.calculate_quantile_mask_2()

        self.total_it = 0
        self.Q_G_eval_interval = Q_G_eval_interval
        self.Q_G_n_per_episode = Q_G_n_per_episode
        self.Q_G_delta = 0
        self.sampling_scheme = sampling_scheme
        self.delta_gamma = delta_gamma
        self.d_update_interval = d_update_interval
        self.use_acc = use_acc
        self.num_critic_updates = num_critic_updates
        if use_acc:
            self.adjusted_dropped_quantiles = torch.tensor(adjusted_dropped_quantiles_init, requires_grad=True)
            self.adjusted_dropped_quantiles_max = adjusted_dropped_quantiles_max
            self.dropped_quantiles_dropped_optimizer = torch.optim.SGD([self.adjusted_dropped_quantiles], lr=lr_dropped_quantiles)
            self.first_training = True
            self.diff_ma_coef = diff_ma_coef

    def calculate_quantile_mask(self):
        top_quantiles_to_drop = self.log_eta.sigmoid() * self.quantiles_total
        self.samples_mask = torch.zeros(1, self.quantiles_total, dtype=torch.float64, device=DEVICE)
        for i in range(self.quantiles_total):
            mask = self.quantiles_total - top_quantiles_to_drop - i
            self.samples_mask[0, i] = max(min(mask, 1), 0)

    def calculate_quantile_mask_2(self):
        top_quantiles_to_drop = self.top_quantiles_to_drop
        top = ceil(top_quantiles_to_drop)
        bot = floor(top_quantiles_to_drop)
        self.samples_mask[0, 0:self.quantiles_total - top] = 1
        self.samples_mask[0, self.quantiles_total - bot:] = 0
        if top != bot:
            self.samples_mask[0, self.quantiles_total - top] = top - top_quantiles_to_drop

    def add_next_z_metrics(self, metrics, next_z):
        for t in range(1, self.critic.n_quantiles + 1):
            total_quantiles_to_keep = t * self.critic.n_nets
            metrics[f'Target_Q/Q_value_t={t}'] = next_z[:, :total_quantiles_to_keep].mean().__float__()

    def train(self, replay_buffer, batch_size=256):
        metrics = dict()
        #alpha = torch.exp(self.log_alpha)
        if ptr_list is not None and do_beta_update:
            self.update_beta(replay_buffer, ptr_list, disc_return)

        for it in range(self.num_critic_updates):
            state, action, next_state, reward, not_done, *_ = replay_buffer.sample(batch_size)
            alpha = torch.exp(self.log_alpha)
            metrics['alpha'] = alpha.item()
            metrics['top_quantiles_to_drop'] = self.top_quantiles_to_drop


            # --- Q loss ---
            with torch.no_grad():
                # get policy action
                new_next_action, next_log_pi = self.actor(next_state)
                # compute and cut quantiles at the next state
                next_z = self.critic_target(next_state, new_next_action)  # batch x nets x quantiles
                sorted_z, _ = torch.sort(next_z.reshape(batch_size, -1))
                self.add_next_z_metrics(metrics, sorted_z)
                if self.use_acc:
                    sorted_z_part = sorted_z[:, :self.quantiles_total]
                else:
                    sorted_z_part = sorted_z[:, :self.quantiles_total]

                # compute target

                target = reward + not_done * self.discount * (sorted_z_part - alpha * next_log_pi)

        cur_z = self.critic(state, action)
        critic_loss = quantile_huber_loss_f(cur_z, target, self.samples_mask)
        metrics['critic_loss'] = critic_loss.item()

        # --- Policy and alpha loss ---
        new_action, log_pi = self.actor(state)
        metrics['actor_entropy'] = - log_pi.mean().item()
        alpha_loss = -self.log_alpha * (log_pi + self.target_entropy).detach().mean()


        # --- Update ---
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        actor_loss = (alpha * log_pi - self.critic(state, new_action).mean(2).mean(1, keepdim=True)).mean()
        metrics['actor_loss'] = actor_loss.item()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.alpha_optimizer.zero_grad()
        alpha_loss.backward()
        self.alpha_optimizer.step()

        if self.total_it > 10000 and (self.total_it + 1) % self.Q_G_eval_interval == 0:
            self.eval_thresholds(replay_buffer, self.Q_G_n_per_episode)

        if self.total_it > 10000 and (self.total_it + 1) % self.d_update_interval == 0:
            self.update_d()

        self.total_it += 1
        return metrics
    def update_beta(self, replay_buffer, ptr_list=None, disc_return=None):
            state, action = replay_buffer.states_by_ptr(ptr_list)
            disc_return = torch.FloatTensor(disc_return).to(DEVICE)
            assert disc_return.shape[0] == state.shape[0]

            mean_Q_last_eps =  self.critic(state, action).mean(2).mean(1, keepdim=True).mean().detach()
            mean_return_last_eps = torch.mean(disc_return).detach()

            if self.first_training:
                    self.diff_mvavg = torch.abs(mean_return_last_eps - mean_Q_last_eps).detach()
                    self.first_training = False
            else:
                    self.diff_mvavg = (1 - self.diff_ma_coef) * self.diff_mvavg \
                                                        + self.diff_ma_coef * torch.abs(mean_return_last_eps - mean_Q_last_eps).detach()

            diff_qret = ((mean_return_last_eps - mean_Q_last_eps) / (self.diff_mvavg + 1e-8)).detach()
            aux_loss = self.adjusted_dropped_quantiles * diff_qret
            self.dropped_quantiles_dropped_optimizer.zero_grad()
            aux_loss.backward()
            self.dropped_quantiles_dropped_optimizer.step()
            self.adjusted_dropped_quantiles.data = self.adjusted_dropped_quantiles.clamp(min=0., max=self.adjusted_dropped_quantiles_max)

    def eval_thresholds_by_type(self, replay_buffer, n_per_episode, sampling_scheme):
        res = dict()
        alpha = torch.exp(self.log_alpha)
        if sampling_scheme == 'uniform':
            states, actions, returns, bs_states, bs_multiplier = replay_buffer.gather_returns_uniform(self.discount, float(alpha), n_per_episode)
        elif sampling_scheme == 'episodes':
            states, actions, returns, bs_states, bs_multiplier = replay_buffer.gather_returns(self.discount, float(alpha), n_per_episode)
        else:
            raise Exception("No such sampling scheme")
        tail_actions = self.actor(bs_states)[0]
        tail_z = self.critic_target(bs_states, tail_actions)
        tail_z = tail_z.reshape(tail_z.shape[0], -1)
        tail_z = tail_z.mean(1, keepdim=True) * bs_multiplier * np.power(replay_buffer.gamma, replay_buffer.q_g_rollout_length)
        res[f'LastReplay_{sampling_scheme}/Returns'] = (returns + tail_z).mean().__float__()

        cur_z = self.critic(states, actions)
        cur_z = cur_z.reshape(cur_z.shape[0], -1)
        cur_z = cur_z.sort(dim=1)[0]
        for t in range(1, self.critic.n_quantiles + 1):
            total_quantiles_to_keep = t * self.critic.n_nets
            res[f'LastReplay_{sampling_scheme}/Q_value_t={t}'] = cur_z[:, :total_quantiles_to_keep].mean().__float__()
        return res

    def eval_thresholds(self, replay_buffer, n_per_episode):
        res_uniform = self.eval_thresholds_by_type(replay_buffer, n_per_episode, 'uniform')
        res_episodes = self.eval_thresholds_by_type(replay_buffer, n_per_episode, 'episodes')
        res = dict()
        res.update(res_uniform)
        res.update(res_episodes)
        last_Q_G_delta = res[f'LastReplay_{self.sampling_scheme}/Q_value_t={self.critic.n_quantiles}'] - \
                         res[f'LastReplay_{self.sampling_scheme}/Returns']
        self.Q_G_delta = self.Q_G_delta * self.delta_gamma + last_Q_G_delta * (1 - self.delta_gamma)
        return res

    def update_d(self):
        if self.Q_G_delta < 0 and self.top_quantiles_to_drop > 0:
            self.top_quantiles_to_drop -= 1
        elif self.Q_G_delta > 0 and self.top_quantiles_to_drop < self.quantiles_total:
            self.top_quantiles_to_drop += 1
        self.calculate_quantile_mask_2()
    def save(self, filename):
        filename = str(filename)
        self.light_save(filename)
        torch.save(self.critic_optimizer.state_dict(), filename + "_critic_optimizer")
        torch.save(self.actor_optimizer.state_dict(), filename + "_actor_optimizer")
        torch.save(self.alpha_optimizer.state_dict(), filename + "_alpha_optimizer")

    def light_save(self, filename):
        filename = str(filename)
        torch.save(self.critic.state_dict(), filename + "_critic")
        torch.save(self.critic_target.state_dict(), filename + "_critic_target")
        torch.save(self.actor.state_dict(), filename + "_actor")
        torch.save(self.log_alpha, filename + "_log_alpha")

    def load(self, filename):
        filename = str(filename)
        self.critic.load_state_dict(torch.load(filename + "_critic"))
        self.critic_target.load_state_dict(torch.load(filename + "_critic_target"))
        self.critic_optimizer.load_state_dict(torch.load(filename + "_critic_optimizer"))
        self.actor.load_state_dict(torch.load(filename + "_actor"))
        self.actor_optimizer.load_state_dict(torch.load(filename + "_actor_optimizer"))
        self.log_alpha = torch.load(filename + '_log_alpha')
        self.alpha_optimizer = torch.optim.Adam([self.log_alpha], lr=3e-4)
        self.alpha_optimizer.load_state_dict(torch.load(filename + "_alpha_optimizer"))  




In [7]:
%%capture

#env = gym.make("BipedalWalker-v3")
#eval_env = gym.make("BipedalWalker-v3")
env = gym.make("BipedalWalkerHardcore-v3") # only attempt this when your agent has solved BipedalWalker-v3
eval_env = gym.make("BipedalWalkerHardcore-v3") 
env = gym.wrappers.Monitor(env, "/workspace/video5", video_callable=lambda ep_id: ep_id%video_every == 0, force=True)

obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

In [8]:
print('The environment has {} observations and the agent can take {} actions'.format(obs_dim, act_dim))
print('The device is: {}'.format(device))

if device.type != 'cpu': print('It\'s recommended to train on the cpu for this')

The environment has 24 observations and the agent can take 4 actions
The device is: cuda
It's recommended to train on the cpu for this


In [9]:
!pip install pybullet_envs

ERROR: Could not find a version that satisfies the requirement pybullet_envs
ERROR: No matching distribution found for pybullet_envs


In [ ]:
2# in the submission please use seed 42 for verification
torch.autograd.set_detect_anomaly(True)
import copy
import datetime
seed = 42
torch.manual_seed(seed)
env.seed(seed)
random.seed(seed)
np.random.seed(seed)
env.action_space.seed(seed)

# logging variables
ep_reward = 0
reward_list = []
plot_data = []
log_f = open("/workspace/agent-log23.txt","w+")
n_quantiles = 25
top_quantiles_to_drop_per_net = 1
n_nets = 5
batch_size = 256
discount = 0.99
tau = 0.005
# initialise agent
Q_G_eval_interval = 50
Q_G_n_episodes = 50
Q_G_n_per_episode =20
Q_G_rollout_length = 500
delta_gamma = 0.99
max_episodes = 200000000
max_timesteps = 2000
d_update_interval = 50000
sampling_scheme = 'episodes'
use_acc = True
import pickle
env = RescaleAction(env, -1., 1.)
eval_env = RescaleAction(eval_env, -1., 1.)


replay_buffer = ReplayBuffer(obs_dim, act_dim, gamma=discount,
                                            n_episodes_to_store=Q_G_n_episodes, q_g_rollout_length=Q_G_rollout_length)
agent = Agent(obs_dim, act_dim).to(device)
critic = Critic(obs_dim, act_dim, n_quantiles, n_nets).to(device)
critic_target = copy.deepcopy(critic)

top_quantiles_to_drop = top_quantiles_to_drop_per_net * n_nets

trainer = Trainer(actor=agent,
                      critic=critic,
                      critic_target=critic_target,
                      top_quantiles_to_drop=top_quantiles_to_drop,
                      discount=discount,
                      tau= tau,
                      target_entropy=-np.prod(env.action_space.shape).item(),sampling_scheme=sampling_scheme,
                      Q_G_eval_interval=Q_G_eval_interval,
                      Q_G_n_per_episode=Q_G_n_per_episode,
                      delta_gamma=delta_gamma,
                      d_update_interval=d_update_interval,                      
                      use_acc=use_acc,
                      lr_dropped_quantiles=0.1,
                      adjusted_dropped_quantiles_init=2.5,
                      adjusted_dropped_quantiles_max=5.0,
                      diff_ma_coef=0.05,
                      num_critic_updates=1)

evaluations = []
state, done = env.reset(), False

total_num_steps = 0
episode_return, last_episode_return = 0, 0
episode_timesteps = 0
episode_num = 0
if use_acc:
    reward_list = []
    start_ptr = replay_buffer.ptr
    ptr_list = []
    disc_return = []
    time_since_beta_update = 0
    do_beta_update = False
agent.train()
beta_udate_rate = 1000
init_num_steps = 25000
size_limit_beta_update_batch = 5000
reward_list2 = []
# training procedure:
for episode in range(1, max_episodes+1):
    for t in range(max_timesteps):
        #print(t)
        total_num_steps += 1
        episode_timesteps += 1
        # select the agent action
        action = agent.sample_action(state)

        # take action in environment and get r and s'
        next_state, reward, done, _ = env.step(action)
        ep_end = done or total_num_steps >= max_timesteps
        replay_buffer.add(state, action, next_state, reward, done, ep_end)
        state = next_state
        ep_reward += reward
        # stop iterating when the episode finished
        if done or t==(max_timesteps-1):
            break
        if use_acc:
            reward_list2.append(ep_reward)
            time_since_beta_update += 1
        if done or episode_timesteps >= max_episodes:
            if use_acc:
                ptr_list.append([start_ptr, replay_buffer.ptr])
                start_ptr = replay_buffer.ptr
                if t > 1:
                    for i in range(episode_timesteps):
                        disc_return.append(
                            np.sum(np.array(reward_list2)[i:] * (discount ** np.arange(0, episode_timesteps - i))))
                    if time_since_beta_update >= beta_udate_rate and t >= init_num_steps:
                        do_beta_update = True
            reward_list2 = []
        # Train agent after collecting sufficient data
        if t >= 10:
            if use_acc and do_beta_update:
                trainer.train(replay_buffer, batch_size, ptr_list, disc_return, do_beta_update)
                do_beta_update= False
                for ii, ptr_pair in enumerate(copy.deepcopy(ptr_list)):
                    if (ptr_pair[0] < replay_buffer.ptr - size_limit_beta_update_batch):
                        disc_return = disc_return[ptr_pair[1] - ptr_pair[0]:]
                        ptr_list.pop(0)
                    elif (ptr_pair[0] > replay_buffer.ptr and
                             replay_buffer.max_size - ptr_pair[0] + replay_buffer.ptr > size_limit_beta_update_batch):
                        if ptr_pair[1] > ptr_pair[0]:
                            disc_return = disc_return[ptr_pair[1] - ptr_pair[0]:]
                        else:
                            disc_return = disc_return[replay_buffer.max_size - ptr_pair[0] + ptr_pair[1]:]
                        ptr_list.pop(0)
                    else:
                        break
                time_since_beta_update = 0
            else:
                trainer.train(replay_buffer, batch_size)

        if done or episode_timesteps >= max_episodes:
            # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
            print(f"Seed: {seed} Total T: {t + 1} Episode Num: {episode_num + 1} Episode T: {episode_timesteps} Reward: {ep_reward:.1f}")
            # Reset environment
            state, done = env.reset(), False

            last_episode_return = episode_return
            episode_return = 0
            episode_timesteps = 0
            episode_num += 1  
    # append the episode reward to the reward list
    reward_list.append(ep_reward)
    print('episode: {}, reward: {}\n'.format(episode, ep_reward))
    # do NOT change this logging code - it is used for automated marking!
    log_f.write('episode: {}, reward: {}\n'.format(episode, ep_reward))
    log_f.flush()
    ep_reward = 0
    state, done = env.reset(), False
    episode += 1    
    # print reward data every so often - add a graph like this in your report
    if episode % plot_interval == 0:
        plot_data.append([episode, np.array(reward_list).mean(), np.array(reward_list).std()])
        reward_list = []
        reward_list2 = []
        # plt.rcParams['figure.dpi'] = 100
        plt.plot([x[0] for x in plot_data], [x[1] for x in plot_data], '-', color='tab:grey')
        plt.fill_between([x[0] for x in plot_data], [x[1]-x[2] for x in plot_data], [x[1]+x[2] for x in plot_data], alpha=0.2, color='tab:grey')
        plt.xlabel('Episode number')
        plt.ylabel('Episode reward')
        plt.show()
        disp.clear_output(wait=True)
 
    if episode%50 == 0:
        trainer_save_name = f'/workspace/iterhard3_{episode}'
        trainer.save(trainer_save_name)
        with open(f'/workspace/iterhard3_{episode}_replay', 'wb') as outF:
            pickle.dump(replay_buffer, outF)
        with open(f'/workspace/plothard3_{episode}_replay', 'wb') as outF:
            pickle.dump(plot_data, outF)  

episode: 760, reward: 67.77401273602067

episode: 761, reward: 112.29566233904774

episode: 762, reward: -54.795982101625455



In [ ]:
import pickle
trainer = Trainer(actor=agent,
                      critic=critic,
                      critic_target=critic_target,
                      top_quantiles_to_drop=top_quantiles_to_drop,
                      discount=discount,
                      tau= tau,
                      target_entropy=-np.prod(env.action_space.shape).item(),sampling_scheme=sampling_scheme,
                      Q_G_eval_interval=Q_G_eval_interval,
                      Q_G_n_per_episode=Q_G_n_per_episode,
                      delta_gamma=delta_gamma,
                      d_update_interval=d_update_interval,                      
                      use_acc=use_acc,
                      lr_dropped_quantiles=0.1,
                      adjusted_dropped_quantiles_init=2.5,
                      adjusted_dropped_quantiles_max=5.0,
                      diff_ma_coef=0.05,
                      num_critic_updates=1)
trainer_save_name = f'/workspace/iterhard3_{episode}'
trainer.save(trainer_save_name)
with open(f'/workspace/iterhard3_{episode}_replay', 'wb') as outF:
    pickle.dump(replay_buffer, outF)
with open(f'/workspace/plothard3_{episode}_replay', 'wb') as outF:
    pickle.dump(plot_data, outF)  